# Call Centre Optimization Exercise - SOLUTIONS 📞

⚠️ **SOLUTIONS:** This notebook contains example solutions for the [experiments exercise](./07a_exercise.ipynb)

**Objective**: Apply your knowledge of the `Experiment` class to analyze call centre performance under different scenarios.

## Learning Goals
- Create `Experiment` objects with different parameter configurations
- Run single experiments and interpret results  
- Conduct multiple replications for statistical reliability
- Compare scenarios and make data-driven recommendations

## Exercise Overview
You will analyze a call centre's performance by testing different staffing levels and demand scenarios. The exercise is divided into 4 tasks.

## 1. Imports 

In [1]:
import numpy as np
import pandas as pd
import simpy
import itertools

# Set display options for 2dp in pandas
pd.set_option('display.precision', 2)

## 2. Simulation model imports

For convenience the model is stored in a python module called `basic_model`.  We need to import:

* The `Experiment` class - to set parameters,
* Functions that wrap the model and allow us to run it. I.e. `single_run` and `multiple_replications`.
* A function to toggle simulation debug info on and off `set_trace`
* A function that will summarise the results of multiple experiments `create_summary_table`

In [2]:
from basic_model import (
    Experiment, 
    single_run, 
    multiple_replications, 
    create_summary_table,
    set_trace,
)

## 4. Exercises

### 4.1 Baseline Analysis

First, let's understand the current system performance.

**Instructions**:
1. Create a default experiment using the `Experiment()` class
2. Run it **once** to see the simulation events. Toggle trace on and off.
3. Record the mean waiting time and operator utilization.
4. Now run 10 multiple replications of the model and view results.
6. Answer the reflection question below


In [3]:
# TODO: Create a default experiment
baseline_experiment = Experiment()

# TODO: toggle trace to False/True
set_trace(trace_on=False)

# TODO: Run the experiment once and store results
baseline_results = single_run(baseline_experiment)

# Display results
print("=== BASELINE SCENARIO RESULTS ===")
print(f"Mean waiting time: {baseline_results['01_mean_waiting_time']:.2f} minutes")
print(f"Operator utilization: {baseline_results['02_operator_util']:.2f}%")

=== BASELINE SCENARIO RESULTS ===
Mean waiting time: 3.87 minutes
Operator utilization: 92.90%


We will now run multiple replications of the model. The function returns a `pandas` dataframe containing a a replication per row and a column per KPI.

In [4]:
# Run multiple replications (trace off!)
set_trace(trace_on=False)

# TODO: Run the experiment once and store results
baseline_reps = multiple_replications(baseline_experiment, n_reps=10)

# show results
baseline_reps

,01_mean_waiting_time,02_operator_util
rep,,
1,3.87,92.90
2,3.20,93.64
3,3.53,94.22
4,1.54,90.90
5,3.07,93.53
6,2.08,93.60
7,2.01,91.18
8,1.82,88.85
9,2.25,93.23


In [5]:
# summary statistics
baseline_reps.describe()

,01_mean_waiting_time,02_operator_util
count,10.00,10.00
mean,2.69,92.69
std,0.84,1.83
min,1.54,88.85
25%,2.03,91.61
50%,2.66,93.38
75%,3.44,93.63
max,3.87,94.85


💡🤔 **Reflection Question**: Based on the baseline results, what do you observe about the system performance? Is the utilization high or low? What might this suggest about the current staffing level?

**Your Answer**: [Write your analysis here]

### 4.2 Staffing Scenarios Analysis

Now test three different staffing scenarios using multiple replications for statistical reliability.

**Scenarios to test**:
1. **Reduced Staffing**: 11 operators
2. **Current Staffing**: 13 operators (baseline)  
3. **Increased Staffing**: 15 operators

**Requirements**:
- Run each scenario with **10 replications**
- Calculate mean and standard deviation for both KPIs
- Create a summary comparison table


In [6]:
# Task 2: Staffing Scenarios
# Turn off tracing for cleaner output
set_trace(trace_on=False)

# TODO: Create three experiments with different staffing levels
reduced_staffing = Experiment(n_operators=11)
current_staffing = Experiment(n_operators=13)  # baseline
increased_staffing = Experiment(n_operators=15)

# TODO: Run 10 replications for each scenario
print("Running reduced staffing scenario (11 operators)...")
reduced_results = multiple_replications(reduced_staffing, n_reps=10)

print("Running current staffing scenario (13 operators)...")
current_results = multiple_replications(current_staffing, n_reps=10)

print("Running increased staffing scenario (15 operators)...")
increased_results = multiple_replications(increased_staffing, n_reps=10)

print("All scenarios completed!")

Running reduced staffing scenario (11 operators)...
Running current staffing scenario (13 operators)...
Running increased staffing scenario (15 operators)...
All scenarios completed!


In [7]:
# TODO: Calculate summary statistics for each scenario

# Create results dictionary
scenario_results = {
    'Reduced (11 ops)': reduced_results,
    'Current (13 ops)': current_results,
    'Increased (15 ops)': increased_results
}

# Create summary table
scenarios_summary = create_summary_table(
    results_dict=scenario_results,
    label_key='Scenario',
    label_order=['Reduced (11 ops)', 'Current (13 ops)', 'Increased (15 ops)']
)

print("=== STAFFING SCENARIOS SUMMARY ===")
scenarios_summary

=== STAFFING SCENARIOS SUMMARY ===


,Scenario,Mean_Waiting_Time,Std_Waiting_Time,Mean_Utilization,Std_Utilization
0,Reduced (11 ops),50.10,11.61,98.95,0.58
1,Current (13 ops),2.69,0.84,92.69,1.83
2,Increased (15 ops),0.47,0.10,80.57,1.61


### 4.3 Demand Variation Analysis

Test how the baseline staffing (13 operators) performs under different demand levels by varying the mean inter-arrival time.

**Demand Scenarios**:
1. **Low Demand**: `mean_iat=0.8` (calls arrive less frequently)
2. **Current Demand**: `mean_iat=0.6` (baseline)
3. **High Demand**: `mean_iat=0.4` (calls arrive more frequently)

**Requirements**:
- Use 10 replications for each demand scenario
- Keep staffing at 13 operators for all tests
- Compare results to identify potential problems


In [8]:
# Task 3: Demand Variation Analysis
# TODO: Create three experiments with different demand levels (mean_iat)
low_demand = Experiment(n_operators=13, mean_iat=0.8)
current_demand = Experiment(n_operators=13, mean_iat=0.6)  # baseline
high_demand = Experiment(n_operators=13, mean_iat=0.4)

# TODO: Run 10 replications for each demand scenario
print("Running low demand scenario (mean_iat=0.8)...")
low_demand_results = multiple_replications(low_demand, n_reps=10)

print("Running current demand scenario (mean_iat=0.6)...")
current_demand_results = multiple_replications(current_demand, n_reps=10)

print("Running high demand scenario (mean_iat=0.4)...")
high_demand_results = multiple_replications(high_demand, n_reps=10)

print("All demand scenarios completed!")

Running low demand scenario (mean_iat=0.8)...
Running current demand scenario (mean_iat=0.6)...
Running high demand scenario (mean_iat=0.4)...
All demand scenarios completed!


In [9]:
# TODO: Create summary table for demand scenarios

# Create results dictionary
scenario_results = {
    'Low (IAT=0.8)': low_demand_results,
    'Current (IAT=0.6)': current_demand_results,
    'High (IAT=0.4)': high_demand_results
}

# Create summary table
demand_summary = create_summary_table(
    results_dict=scenario_results,
    label_key='Scenario',
    label_order=['Low (IAT=0.8)', 'Current (IAT=0.6)', 'High (IAT=0.4)']
)

print("=== STAFFING SCENARIOS SUMMARY ===")
demand_summary

=== STAFFING SCENARIOS SUMMARY ===


,Scenario,Mean_Waiting_Time,Std_Waiting_Time,Mean_Utilization,Std_Utilization
0,Low (IAT=0.8),0.19,0.05,69.44,1.96
1,Current (IAT=0.6),2.69,0.84,92.69,1.83
2,High (IAT=0.4),143.34,9.91,99.28,0.15


### 4.4 Recommendations

Based on your analysis from the exercises above, provide data-driven recommendations.

**Questions to address**:

1. **Optimal Staffing**: What is the "optimal" number of operators for current demand levels? Consider both waiting times and utilization efficiency.

2. **High Demand Response**: What happens to the system under high demand conditions? What would you recommend to management?

3. **Trade-offs**: Explain the trade-off between operator utilization and customer waiting times that you observed.

**Your Recommendations**:

### Question 1 - Optimal Staffing
[Provide your analysis and recommendation here]

### Question 2 - High Demand Response  
[Describe what happens under high demand and your recommendations]

### Question 3 - Trade-offs
[Explain the utilization vs. waiting time trade-off you observed]
